<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Papers/blob/main/Violence_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import glob
from IPython.display import clear_output 

#Datasets

In [2]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset
!unzip \*.zip && rm *.zip
clear_output()

In [ ]:
!git clone https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos.git

Cloning into 'A-Dataset-for-Automatic-Violence-Detection-in-Videos'...
remote: Enumerating objects: 376, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 376 (delta 3), reused 11 (delta 3), pack-reused 364
Receiving objects: 100% (376/376), 1.02 GiB | 38.57 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
os.makedirs('/content/NonViolence/', exist_ok=True)
os.makedirs('/content/Violence/', exist_ok=True)

nv = '/content/Real Life Violence Dataset/NonViolence/'
v = '/content/Real Life Violence Dataset/Violence/'

for V in [v, nv]:
  for i in os.listdir(V):
    R = V+i
    os.replace(R, R.replace('Real Life Violence Dataset/', ''))

In [ ]:
path = '/content/A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset/'
choices = ['/content/NonViolence/', '/content/NonViolence/', '/content/Violence/', '/content/Violence/']

for id, V in enumerate(['non-violent/cam1/', 'non-violent/cam2/', 'violent/cam1/', 'violent/cam2/']):
  for i in os.listdir(path+V):
    os.replace(path+V+i, choices[id]+i)

In [ ]:
os.makedirs('/content/data/', exist_ok=True)
os.replace('/content/NonViolence/', '/content/data/NonViolence/')
os.replace('/content/Violence/', '/content/data/Violence/')

#2

In [ ]:
!pip uninstall emoji
!pip install emoji==1.7.0
!pip install text2emotion

import text2emotion as te

clear_output()

In [4]:
sample = "I was asked to sign a third party contract a week out from stay"
te.get_emotion(sample)

{'Happy': 0.0, 'Angry': 0.25, 'Surprise': 0.0, 'Sad': 0.0, 'Fear': 0.75}